In [ ]:
import numpy as np
import pandas as pd
import os
import datetime
import warnings
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt
import pandas_ta as ta
import pandas as pd
import yfinance as yf
import numpy as np
import math
import statistics
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import urllib.request, json
from pandas import json_normalize
import math
 # The first thing we want to do is import the Pandas library and set the filepath to our data file
import pandas as pd
import pandas_ta as ta
import warnings
from pandas.tseries.offsets import DateOffset
import warnings
import sklearn

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import groupby, chain
from collections import OrderedDict

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import quandl
quandl.ApiConfig.api_key = 'soxd-469x3Zp4ib_4uzc'


##########################

def correlation_study(df_close, df, date_column, value_column, close_column):
    # join both datasets together
    together = pd.merge(df_close[[date_column, close_column]],
                        df[[date_column, value_column]],
                        on= [date_column], how='left')
    # last valid observation forward
    together = together.fillna(method='ffill')
    # drop NAs
    together = together.dropna(axis=0)
    return together.corr()[value_column].iloc[0]

df_close = df_MKPRU.copy()
df_close = df_close.reset_index()
df_close = df_close.rename(columns={'Value': 'Close'})
# PARAMETERS #
date_column = 'Date'
close_column = 'Close'
value_column = 'Value'
list_of_hash_df = [df_CPTRA, df_DIFF, df_ETRAV, df_ETRVU, df_HRATE, df_MIREV, df_MKPRU, df_MKTCP, df_MWNTD, \
                   df_MWNUS,  df_NADDU, df_NTRAN, df_NTRAT, df_NTRBL, df_NTREP, df_TOTBC, df_TOUTV, df_TRFEE, \
                   df_TRFUS, df_TRVOU, df_AVBLS, df_BLCHS, df_ATRCT, df_MIREV]
list_of_hash_names= [ 'df_CPTRA', 'df_DIFF', 'df_ETRAV', 'df_ETRVU', 'df_HRATE','df_MIREV', 'df_MKPRU', \
                     'df_MKTCP', 'df_MWNTD', 'df_MWNUS', 'df_NADDU', 'df_NTRAN', 'df_NTRAT', 'df_NTRBL', \
                     'df_NTREP', 'df_TOTBC', 'df_TOUTV', 'df_TRFEE', 'df_TRFUS', 'df_TRVOU', 'df_AVBLS', 'df_BLCHS', \
                     'df_ATRCT', 'df_MIREV']
zipped_list = list(zip(list_of_hash_names, list_of_hash_df))

# CORRELATIONS ITERATION #
list_of_correlations = []
for i, j in zipped_list:
    df_hash_features = j.copy()
    df_hash_features = df_hash_features.reset_index()
    corr = correlation_study(df_close, df_hash_features, date_column, value_column, close_column)
    print(i, corr)


##########################

def get_data_yfinance(asset, interval='1d', start='2011-01-01'):
    date_column = 'Date'
    list_of_days = ['1d', '5d', '1wk', '1mo', '3mo']
    list_of_minutes = ['1m', '2m', '5m', '15m', '30m', '60m', '90m']
    df = yf.download(asset, start=start, threads= False, interval=interval)
    df = df.reset_index()
    if interval== '1h':
        df = df.rename(columns={'index':'Date'})
    elif interval in list_of_minutes:
        df = df.rename(columns={'Datetime':'Date'})
        #df[date_column] = df[date_column].str[:]    
    # Now that we have loaded our data into the dataframe, we can preview it using the print & .head() function
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values([date_column], ascending=[True]).reset_index(drop=True).sort_values([date_column], ascending=[False])
    df = df.rename(columns={'Date':'date','Open':'open', 'High':'high', 'Low': 'low','Close':'close', 'Volume':'tradecount'})
    return(df)

# Create a Dataframe with dates
first_date = '2009-01-02'
first_date_obj = datetime.strptime(first_date, '%Y-%m-%d')
today_obj = datetime.today()
difference = today_obj - first_date_obj
total_days = difference.days
list_of_dates = []
for i in range(total_days):
    new_day = first_date_obj + timedelta(days=i)
    new_day = new_day.strftime('%Y-%m-%d')
    list_of_dates.append(new_day)
df_dates = pd.DataFrame(list_of_dates, columns=['date'])
df_dates['date'] = pd.to_datetime(df_dates['date'], format='%Y-%m-%d')

df_close_yf = get_data_yfinance('BTC-USD')
df_close_yf = df_close_yf[['date','close']]
df_MKPRU_quandul = df_MKPRU.reset_index().rename(columns={'Date':'date', 'Value':'close'})
df_close_clean = pd.merge(df_dates,df_MKPRU_quandul, on='date', how='left')
df_close_clean = pd.merge(df_close_clean,df_close_yf, on='date', how='left')
df_close_clean['close'] = df_close_clean[['close_x','close_y']].apply(lambda x: x['close_x'] if(np.all(pd.notnull(x['close_x']))) else x['close_y'], axis=1)
df_close_clean = df_close_clean[['date','close']]


##########################


class target:
    def __init__(self,df,column_date='date'):
        self.df = df
        self.column_date = column_date
       
    def max_min(self, initial_days=0, days=10, column_target='high', method=max):
        df = self.df
        column_date = self.column_date

        df_target = df[[column_date,column_target]].copy()
        df_target_original = df_target.copy()
        list_of_targets=[column_date]
        #in loop add one column per day
        for i in range(initial_days,days+1):
            df_target_temp = df_target_original.copy()
            df_target_temp[column_date+str(i)] = df_target_temp[column_date] + pd.Timedelta(days=-i)
            df_target_temp = df_target_temp[[column_date+str(i),column_target]].rename(columns={column_target:column_target+str(i)})
            df_target=pd.merge(df_target,df_target_temp, how='left',left_on=column_date, right_on=column_date+str(i))
            df_target = df_target.dropna()
            list_of_targets.append(column_target+str(i))
       
        df_target = df_target[list_of_targets]
        if method==max:
            df_target['target'] = df_target[list_of_targets].max(axis=1)
        if method==min:
            df_target['target'] = df_target[list_of_targets].min(axis=1)
        df_target=df_target[[column_date, 'target']]
        df_target = df_target.sort_values([column_date], ascending=[True]).reset_index(drop=True).sort_values([column_date], ascending=[False])
       
        return(df_target)  
   
    def close(self, days=10,column_target='close'):
        df = self.df
        column_date = self.column_date    
        df_target = df[[column_date,column_target]].copy()
        df_target[column_date+ "_-_" +str(days)] = df_target[column_date] + pd.Timedelta(days=-days)
        df_target = df_target.drop(columns=column_date, axis=1)
        dictionary={column_date+ "_-_" +str(days):column_date, column_target:'target'}
        df_target = df_target.rename(columns=dictionary)
        df_target = df_target[[column_date, 'target']]
        return (df_target)
   
    def direction(self,days=10,column_target='close', column_high='high', column_low='low'):
        df = self.df
        column_date = self.column_date
        df_close = df[[column_date, column_target]].copy()
        df_target_max = target(df).max_min(method=max, column_target=column_high).rename(columns={'target':'max'})
        df_target_min = target(df).max_min(method=min, column_target=column_low).rename(columns={'target':'min'})
        df_target_close = target(df).close(column_target=column_target).rename(columns={'target':'target_close'})
        df_final = pd.merge(df_close, df_target_max, on=column_date, how='inner')
        df_final = pd.merge(df_final, df_target_min, on=column_date, how='inner')
        df_final = pd.merge(df_final, df_target_close, on=column_date, how='inner')
        df_final['direction'] =df_final[column_target] + (df_final['max'] - df_final[column_target]  + df_final['min'] - df_final[column_target] + df_final['target_close'] - df_final[column_target])/3
        df_final = df_final[['date','direction']]
        return(df_final)

df = df_close_clean.copy()

df_direction = target(df).direction(days=7, column_target='close', column_high='close', column_low='close')
df = pd.merge(df,df_direction, on='date', how='left')
df['difference'] = (df['direction'] - df['close'])/df['close']
df['target_class']=df["difference"].apply(lambda x: 1 if x>0. else 0)
#df = df[['date','open','high','low','close', 'target_class']]
df_target = df[['date', 'close','target_class']]


##########################

# DATA PREPARATION

# To start again, creation of unique df
list_of_fetures_df = [
df_MWNUS,\
df_HRATE,\
df_DIFF,\
df_MKPRU,\
df_MIREV,\
df_CPTRA,\
df_TRVOU,\
df_ETRVU,\
df_ETRAV,\
df_TOUTV,\
df_NTRBL,\
df_NTRAT,\
df_NADDU,\
df_NTREP,\
df_NTRAN,\
df_TRFUS,\
df_TRFEE,\
df_MKTCP,\
df_TOTBC,\
df_AVBLS,\
df_BLCHS,\
df_ATRCT]

list_of_names_df = ['df_MWNUS',\
'df_HRATE',\
'df_DIFF',\
'df_MKPRU',\
'df_MIREV',\
'df_CPTRA',\
'df_TRVOU',\
'df_ETRVU',\
'df_ETRAV',\
'df_TOUTV',\
'df_NTRBL',\
'df_NTRAT',\
'df_NADDU',\
'df_NTREP',\
'df_NTRAN',\
'df_TRFUS',\
'df_TRFEE',\
'df_MKTCP',\
'df_TOTBC',\
'df_AVBLS',\
'df_BLCHS',\
'df_ATRCT']

df_final = df_target.copy()
zipped_corr_list = list(zip(list_of_fetures_df, list_of_names_df))
for i, j in zipped_corr_list:
    i = i.reset_index().rename(columns={'Date':'date', 'Value':j})
    df_final = pd.merge(df_final,i, on='date', how='left')
#df_rf = df_final.drop(columns=['date'], axis=1)
df_MWNTD = df_MWNTD.reset_index().rename(columns={'Date':'date', 'Value':'df_MWNTD'})
df_MWNTD['date'] = pd.DatetimeIndex(df_MWNTD.date) + pd.DateOffset(1)
#df_MWTRV = df_MWTRV.reset_index().rename(columns={'Date':'date', 'Value':'df_MWTRV'})
#df_MWTRV['date'] = pd.DatetimeIndex(df_MWTRV.date) + pd.DateOffset(1)
df_final = pd.merge(df_final,df_MWNTD, on='date', how='left')
#df_final = pd.merge(df_final,df_MWTRV, on='date', how='inner')
#df_final = df_final.drop(columns='index')
list_of_fetures_df = [
df_MWNUS,\
df_HRATE,\
df_DIFF,\
df_MKPRU,\
df_MIREV,\
df_CPTRA,\
df_TRVOU,\
df_ETRVU,\
df_ETRAV,\
df_TOUTV,\
df_NTRBL,\
df_NTRAT,\
df_NADDU,\
df_NTREP,\
df_NTRAN,\
df_TRFUS,\
df_TRFEE,\
df_MKTCP,\
df_TOTBC,\
df_AVBLS,\
df_BLCHS,\
df_ATRCT,\
df_MWNTD]

list_of_names_df = ['df_MWNUS',\
'df_HRATE',\
'df_DIFF',\
'df_MKPRU',\
'df_MIREV',\
'df_CPTRA',\
'df_TRVOU',\
'df_ETRVU',\
'df_ETRAV',\
'df_TOUTV',\
'df_NTRBL',\
'df_NTRAT',\
'df_NADDU',\
'df_NTREP',\
'df_NTRAN',\
'df_TRFUS',\
'df_TRFEE',\
'df_MKTCP',\
'df_TOTBC',\
'df_AVBLS',\
'df_BLCHS',\
'df_ATRCT',\
'df_MWNTD']

def calculate_sublists(list_of_nulls):
    iterator = 0
    list_of_lists_of_index = []
    list_small_index = []
    check = list_of_nulls[0] - iterator
    for i in range(len(list_of_nulls)):
        value = list_of_nulls[i]
        if check != value-iterator:
            list_of_lists_of_index.append(list_small_index)
            list_small_index = []
            iterator=0
            check = list_of_nulls[i]
            list_small_index.append(value)
        else:
            list_small_index.append(value)
        iterator+=1
    list_of_lists_of_index.append(list_small_index)
    return(list_of_lists_of_index)

def add_average_to_index_list(df_in, field, list_of_nulls):
    df = df_in.copy()
    list_of_lists = calculate_sublists(list_of_nulls)
    for list_ in list_of_lists:
        if list_[0] == 0:
            #take the last_value
            average = df[[field]].iloc[list_[-1]+1]
        else:
            average = (df[[field]].iloc[list_[0]-1] + df[[field]].iloc[list_[-1]+1]) / 2
        for i in list_:
            df.iloc[i, df.columns.get_loc(field)] = average[0]
    return df

df_cleaned = df_final.copy()
for feature in list_of_names_df:
    list_nulos=df_cleaned[df_cleaned[feature].isna()].index
    if len(list_nulos) > 0:
        df_cleaned = add_average_to_index_list(df_cleaned, feature, list_nulos)
    else:
        print("probblem with ", feature)
df_final=df_cleaned.copy()


##########################

# obtain ribbons
list_of_sma = []
list_of_ribbon = []
for i in list_of_names_df:
    sma_5 = i + '_sma_5'
    list_of_sma.append(sma_5)
    df_final[sma_5] = pd.Series.to_frame(ta.sma(df_final[i], length=5))
    sma_10 = i + '_sma_10'
    list_of_sma.append(sma_10)
    df_final[sma_10] = pd.Series.to_frame(ta.sma(df_final[i], length=10))
    sma_20 = i + '_sma_20'
    list_of_sma.append(sma_20)
    df_final[sma_20] = pd.Series.to_frame(ta.sma(df_final[i], length=20))
    sma_30 = i + '_sma_30'
    list_of_sma.append(sma_30)
    df_final[sma_30] = pd.Series.to_frame(ta.sma(df_final[i], length=30))
    sma_40 = i + '_sma_40'
    list_of_sma.append(sma_40)
    df_final[sma_40] = pd.Series.to_frame(ta.sma(df_final[i], length=40))
    sma_60 = i + '_sma_60'
    list_of_sma.append(sma_60)
    df_final[sma_60] = pd.Series.to_frame(ta.sma(df_final[i], length=60))
    ribbon_5_10 = 'ribbon_' + i + '_5_10'
    list_of_ribbon.append(ribbon_5_10)
    df_final[ribbon_5_10] = (df_final[sma_5] - df_final[sma_10])/df_final[sma_10]
    ribbon_10_20 = 'ribbon_' + i + '_10_20'
    list_of_ribbon.append(ribbon_10_20)
    df_final[ribbon_10_20] = (df_final[sma_10] - df_final[sma_20])/df_final[sma_20]
    ribbon_20_40 = 'ribbon_' + i + '_20_40'
    list_of_ribbon.append(ribbon_20_40)
    df_final[ribbon_20_40] = (df_final[sma_20] - df_final[sma_40])/df_final[sma_40]
    ribbon_30_60 = 'ribbon_' + i + '_30_60'
    list_of_ribbon.append(ribbon_30_60)
    df_final[ribbon_30_60] = (df_final[sma_30] - df_final[sma_60])/df_final[sma_60]

list_of_date = ['date']
list_of_sma_to_calculate_inc = list_of_date + list_of_sma
list_of_ribbon_to_calculate_inc = list_of_date + list_of_ribbon
df_sma_to_calculate_inc = df_final[list_of_sma_to_calculate_inc].copy()
df_ribbon_to_calculate_inc = df_final[list_of_ribbon_to_calculate_inc].copy()
df_final.to_csv('datos_hash.csv', encoding='Latin', index=False, sep=';', decimal=',')

# Create previous data
def creation_df_before(df_final, days, date_column='date', list_of_columns_to_delete=['target_class', 'close']):
    df_final_before = df_final.copy()
    df_final_before = df_final_before.drop(columns=list_of_columns_to_delete)
    df_final_before[date_column] = pd.DatetimeIndex(df_final_before.date) + pd.DateOffset(days)
    dictionary_change_columns = {}
    list_columns_df_final_before = []
    for i in df_final_before.columns:
        dictionary_change_columns[i] = 'before_1_day_' + str(i)
        list_columns_df_final_before.append('before_1_day_' + str(i))
    df_final_before = df_final_before.rename(columns=dictionary_change_columns)
    return df_final_before

def incremental_calculation(df, *args, days=1, date_column= 'date', columns_to_drop=['target_class', 'close']):
    df_final_before = creation_df_before(df_final, days)
    df_final_before = df.copy()
    df_final_before = df_final_before.drop(columns=columns_to_drop)
    df_final_before[date_column] = pd.DatetimeIndex(df_final_before.date) + pd.DateOffset(days)
    dictionary_change_columns = {}
    if len(args) == 0:
        list_columns_df_final_before = []
        for i in df_final_before.columns:
            dictionary_change_columns[i] = 'before_' + str(days) + '_days_' + str(i)
            if i != date_column:
                list_columns_df_final_before.append('before_' + str(days) + '_days_' + str(i))
        df_final_before = df_final_before.rename(columns=dictionary_change_columns)
        df_final_before = df_final_before.rename(columns={'before_' + str(days) + '_days_' + 'date': 'date'})
    else:
        list_columns_df_final_before = []
        for n in args:
            list_columns_df_final_before.append(n)
        df_final_before = df_final_before[list_columns_df_final_before]
        for i in df_final_before.columns:
            dictionary_change_columns[i] = 'before_' + str(days) + '_days_' + str(i)
            if i != date_column:
                list_columns_df_final_before.append('before_' + str(days) + '_days_' + str(i))
        df_final_before = df_final_before.rename(columns=dictionary_change_columns)
        df_final_before = df_final_before.rename(columns={'before_' + str(days) + '_days_' + 'date': 'date'})
    return(df_final_before, list_columns_df_final_before)

df_sma_before_1_days,list_columns_df_sma_before_1_days = incremental_calculation(df_sma_to_calculate_inc, days=1, columns_to_drop=[])
df_sma_before_4_days,list_columns_df_sma_before_4_days = incremental_calculation(df_sma_to_calculate_inc, days=4, columns_to_drop=[])
df_sma_before_10_days,list_columns_df_sma_before_10_days = incremental_calculation(df_sma_to_calculate_inc, days=10, columns_to_drop=[])
df_ribbon_before_1_days,list_columns_df_ribbon_before_1_days = incremental_calculation(df_ribbon_to_calculate_inc, days=1, columns_to_drop=[])
df_ribbon_before_4_days,list_columns_df_ribbon_before_4_days = incremental_calculation(df_ribbon_to_calculate_inc, days=4, columns_to_drop=[])
df_ribbon_before_10_days,list_columns_df_ribbon_before_10_days = incremental_calculation(df_ribbon_to_calculate_inc, days=10, columns_to_drop=[])

df_final = pd.merge(df_final, df_sma_before_1_days, on='date', how='left')
df_final = pd.merge(df_final, df_sma_before_4_days, on='date', how='left')
df_final = pd.merge(df_final, df_sma_before_10_days, on='date', how='left')
df_final = pd.merge(df_final, df_ribbon_before_1_days, on='date', how='left')
df_final = pd.merge(df_final, df_ribbon_before_4_days, on='date', how='left')
df_final = pd.merge(df_final, df_ribbon_before_10_days, on='date', how='left')


##########################

# Creation of incrementales
def get_incremental_value(df_in, list_of_features, days):
    df = df_in.copy()
    previous = 'before_' + str(days) + '_days_'
    inc = 'inc_' + str(days) + '_days_'
    list_of_incremental_values = []
    for i in list_of_features:
        df[inc + str(i)] = 100 * (df[i] - df[previous + str(i)]) / df[previous + str(i)]
    return df
df_total_sma = get_incremental_value(df_final, list_of_sma, 1)
df_total_sma = get_incremental_value(df_total_sma, list_of_sma, 4)
df_total_sma = get_incremental_value(df_total_sma, list_of_sma, 10)
df_total_sma_ribbon = get_incremental_value(df_total_sma, list_of_ribbon, 1)
df_total_sma_ribbon = get_incremental_value(df_total_sma_ribbon, list_of_ribbon, 4)
df_total_sma_ribbon = get_incremental_value(df_total_sma_ribbon, list_of_ribbon, 10)

# Delete columns unuseful

list_of_columns_to_delete = []
list_of_list_to_delete = [list_of_sma, list_of_names_df, list_of_sma_to_calculate_inc, list_columns_df_sma_before_1_days, \
                          list_columns_df_sma_before_4_days, list_columns_df_sma_before_10_days, list_columns_df_ribbon_before_1_days, \
                          list_columns_df_ribbon_before_4_days, list_columns_df_ribbon_before_10_days]
for i in list_of_list_to_delete:
    for j in i:
        if j != 'date':
            list_of_columns_to_delete.append(j)

df = df_total_sma_ribbon.drop(columns=list_of_columns_to_delete).dropna()


##########################


def sma_long_and_short_calculation(df_final, indicator, small_sma, big_sma, date_column='date', close_column='close'):
    df_calculo = df_final[[date_column, \
                           'df_'+indicator, \
                           'df_'+ indicator +'_sma_' + str(small_sma), \
                           'df_'+indicator + '_sma_' + str(big_sma), \
                           'before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma), \
                           'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma), \
                           close_column]]
    # 2  long y short signals calculation
    df_calculo["long_signal"] = df_calculo[['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma),\
                                            'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)\
                                           ]].apply(lambda x: 1 if x['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma)]<0 and x['ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]>=0  else 0, axis=1)
   
    df_calculo["short_signal"] = df_calculo[['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma),\
                                             'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]].apply(lambda x: 1 if x['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma)]>0 and x['ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]<=0  else 0, axis=1)
    # 3 calculation maximum, minimum, differences
    list_of_operations = []
    list_of_entry_price = []
    list_of_exit_price = []
    list_of_date_entry=[]
    list_of_date_exit=[]
    list_of_maximum=[]
    list_of_minimum=[]

    maximum_value=0
    minimum_value=0
    entry_long_value=0
    entry_short_value=0
   
    date_enter_short = df_calculo.iloc[0][date_column]
    date_enter_long = df_calculo.iloc[0][date_column]
   
    for i in range(len(df_calculo)):
        if df_calculo.iloc[i]['short_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            #creation new values in long list
            entry_short_value = df_calculo.iloc[i][close_column]
            date_enter_short = df_calculo.iloc[i][date_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            minimum_value = df_calculo.iloc[i][close_column]
            maximum_value = df_calculo.iloc[i][close_column]
            if entry_short_value>0:
                list_of_operations.append('long')
                list_of_entry_price.append(entry_long_value)
                list_of_exit_price.append(entry_short_value)
                list_of_date_entry.append(date_enter_long)
                list_of_date_exit.append(date_enter_short)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)

           
        elif df_calculo.iloc[i]['long_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            date_enter_long = df_calculo.iloc[i][date_column]
            entry_long_value = df_calculo.iloc[i][close_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            maximum_value = df_calculo.iloc[i][close_column]
            minimum_value = df_calculo.iloc[i][close_column]
            if entry_short_value>0:
                list_of_operations.append('short')
                list_of_entry_price.append(entry_short_value)
                list_of_exit_price.append(entry_long_value)
                list_of_date_entry.append(date_enter_short)
                list_of_date_exit.append(date_enter_long)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)

       
        if df_calculo.iloc[i][close_column] < minimum_value:
            minimum_value = df_calculo.iloc[i][close_column]
        elif df_calculo.iloc[i][close_column] > maximum_value:
            maximum_value = df_calculo.iloc[i][close_column]
    data = pd.DataFrame(columns = ['indicator', 'operation', 'entry_price', 'exit_price', 'maximum_price', 'minimum_price', 'entry_date' ,'exit_date'])
    df_out = data.append(pd.DataFrame({'indicator':indicator, 'operation':list_of_operations, 'entry_price':list_of_entry_price, 'exit_price':list_of_exit_price, 'maximum_price':list_of_maximum, 'minimum_price':list_of_minimum, 'entry_date':list_of_date_entry ,'exit_date':list_of_date_exit}), ignore_index = True)

    return(df_out)



##########################

list_of_indicators_to_sma_sin_calculation = [\
'MWNUS',\
'HRATE',\
'DIFF',\
'MKPRU',\
'MIREV',\
'CPTRA',\
'TRVOU',\
'ETRVU',\
'ETRAV',\
'TOUTV',\
'NTRBL',\
'NTRAT',\
'NADDU',\
'NTREP',\
'NTRAN',\
'TRFUS',\
'TRFEE',\
'MKTCP',\
'TOTBC',\
'AVBLS',\
'BLCHS',\
'ATRCT']
df_out = sma_long_and_short_calculation(df_final, 'CPTRA', 30, 60)


##########################


final_data = pd.DataFrame(columns = ['indicator', 'operation', 'entry_price', 'exit_price', 'maximum_price', \
                                     'minimum_price', 'entry_date' ,'exit_date'])
for indicator_ in list_of_indicators_to_sma_sin_calculation:
    new_df = sma_long_and_short_calculation(df_final, indicator_, 30, 60)
    df_out = pd.concat([df_out,new_df],axis=0)
df_out.to_csv('longs_and_short_signals.csv', encoding='Latin', sep=';', decimal=',')